In [1]:
# This will clone and install the branch that has the current changes on it. If you want to see the output of the install to be sure it 
# was successfull, remove the > /dev/null 2>&1 in the second line.  

#!pip install -e astrohack/ > /dev/null 2>&1
#!pip install -e .. > /dev/null 2>&1

In [2]:
import os
import json
import astrohack
import graphviper

import numpy as np
import pandas as pd

import graphviper.utils.logger as logger

In [3]:
os.environ["PARAMETER_CONFIG_PATH"]

'/users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/graphviper/config/:/export/home/fornax/Development/astrohack-issue-246/src/astrohack/config/'

In [4]:
import distributed

from graphviper.dask.client import local_client

DEFAULT_DASK_ADDRESS="tcp://localhost:8786"

log_params = {
    'logger_name': "astrohack",
    'log_level':'INFO',
    'log_to_term':True,
    'log_to_file': False,
}

worker_log_params = {
    'logger_name': "astrohack",
    'log_level':'INFO',
    'log_to_term':True,
    'log_to_file': False,
    'log_file': None
}

client=local_client(
    cores=2,
    memory_limit='8GB',
    log_params=log_params,
    worker_log_params=worker_log_params,
    serial_execution=False
)

[2024-04-24 09:14:20,894]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-04-24 09:14:20,894]     INFO  graphviper:  Module path: /users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10//site-packages/ 
[2024-04-24 09:14:20,895]     INFO  graphviper:  Searching /users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/graphviper/config/ for configuration file, please wait ... 
[2024-04-24 09:14:20,899]  WARNING   astrohack:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 


/users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35987 instead
  warnings.warn(


[2024-04-24 09:14:22,024]     INFO   astrohack:  Created client <MenrvaClient: 'tcp://127.0.0.1:39929' processes=2 threads=2, memory=11.18 GiB> 


In [5]:
client.dashboard_link

'http://127.0.0.1:35987/status'

In [21]:
graphviper.utils.data.download('ea25_cal_small_before_fixed.split.ms', folder="data")

#ms_file = "data/ea25_cal_small_before_fixed.split.ms"
ms_file_before = "/.lustre/cv/users/jhoskins/panel-tests/otf_holo_ka.60362.21125040509.ms"      # before
ms_file_after = "/.lustre/cv/users/jhoskins/panel-tests/otf_holo_ka_000.60380.43906070602.ms"   # after


#point_name = "data/ea25_cal_small_before_fixed.split.point.zarr"
#holog_name = "data/ea25_cal_small_before_fixed.split.holog.zarr"
#image_name = "data/ea25_cal_small_before_fixed.split.image.zarr"
#panel_name = "data/ea25_cal_small_before_fixed.split.panel.zarr"

[2024-04-24 09:50:11,041]     INFO   astrohack:  File exists: data/ea25_cal_small_before_fixed.split.ms 


In [102]:
from astrohack.extract_pointing import extract_pointing
from astrohack.extract_holog import extract_holog
from astrohack.holog import holog

class Pipeline:
    def __init__(self):
        self.ms_name = []
        self.base_name = "data/grid_search"
        
        self.function_parameters = None
        self.search_parameters = None

        self.build_basename()

    def build_basename(self):        
        self.point_basename = ".".join( (self.base_name, "point") )
        self.holog_basename = ".".join( (self.base_name, "holog") )
        self.image_basename = ".".join( (self.base_name, "image") )
        self.panel_basename = ".".join( (self.base_name, "panel") )
        
    
    def run(self, ms_name, tag=".zarr"):
        self.ms_name.append(ms_name)
        
        extract_pointing(
            ms_name=ms_name,
            point_name = ".".join((self.point_basename, tag)),
            **self.function_parameters["extract_pointing"]
        )
            
        extract_holog(
            ms_name=ms_name,
            point_name = ".".join((self.point_basename, tag)),
            holog_name = ".".join((self.holog_basename, tag)),
            **self.function_parameters["extract_holog"]
        )
            
        holog(
            holog_name = ".".join((self.holog_basename, tag)),
            image_name = ".".join((self.image_basename, tag)),
            **self.function_parameters["holog"]
        )

    def calculate_difference(self):
        from astrohack.dio import open_panel

        M_TO_MILS = 39370.1
        panel_list = ["3-16", "5-36", "4-6"]
        
        before_mds = open_panel(".".join((self.panel_basename, "before.zarr")))
        after_mds = open_panel(".".join((self.panel_basename,  "after.zarr")))
        
        before_values = before_mds["ant_ea16"]["ddi_1"].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
        after_values =  after_mds["ant_ea16"]["ddi_1"].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS

        return (after_values - before_values)        

    def calculate_error(self, difference):
        truth = np.array([
            [15., 15., -15., -15.],
            [-20., -20., -20., -20.,],
            [20., 20., 20., 20]
        ])

        N = truth.flatten().shape[0]
        
        return np.power(difference - truth, 2).sum()/(N - 1)

    
    
    def grid_search(self, n):
        grid_dataframe = {
            #"ms": []
        }
        
        for i in range(n):
            #grid_dataframe["ms"].append(self.function_parameters["extract_holog"]["ms_name"])
            
            for key, value in self.search_parameters.items():
                index = np.random.randint(len(value))
                self.function_parameters["panel"][key] = value[index]
                
                if not key in grid_dataframe.keys():
                    grid_dataframe[key] = []
                    
                grid_dataframe[key].append(value[index])
                
            for tag in ["before.zarr", "after.zarr"]:
                panel(
                    panel_name = ".".join((self.panel_basename, tag)),
                    image_name = ".".join((self.image_basename, tag)),
                    **self.function_parameters["panel"]
                )

            difference = self.calculate_difference()
            print(difference)
            for i, panel_ in enumerate(["3-16", "5-36", "4-6"]):
                for j in range(4):
                    key = ".".join((panel_, f"{j}"))
                    if not key in grid_dataframe.keys():
                        grid_dataframe[key] = []

                    grid_dataframe[key].append(difference[i][j])
            
            grid_dataframe["results"] = self.calculate_error(difference)

        df = pd.DataFrame(data=grid_dataframe)
        df.to_csv("grid_search.csv", sep=",", index=False)

In [103]:
pipeline = Pipeline()
pipeline.function_parameters = {
    "extract_pointing": {
        #"ms_name": None,
        "parallel": True,
        "overwrite": True
    },
    "extract_holog": {
        #"ms_name": None,
        "ddi": [1],
        "baseline_average_nearest": 1,
        "parallel": True,
        "overwrite": True
    },
    "holog": {
        "ant": "ea16",
        "ddi": [1],
        "parallel": True,
        "overwrite": True
    },
    "panel": {
        "parallel": True,
        "overwrite": True
    }
}

pipeline.search_parameters = {
    "clip_type": [
        "relative", 
        "sigma"
    ],
    "clip_level": [
        2., 
        3., 
        4.
    ],
    "panel_model": [
        "mean", 
        "rigid", 
        "corotated_scipy", 
        "corotated_lst_sq", 
        "corotated_robust", 
        "xy_paraboloid", 
        "rotated_paraboloid", 
        "full_paraboloid_lst_sq"
    ],
    "panel_margins": [
        0.0, 
        0.05, 
        0.075
    ],
}


In [47]:
files = [ms_file_before, ms_file_after]
tags = ["before.zarr", "after.zarr"]

for args in zip(files, tags):
    pipeline.run(*args)

[2024-04-24 10:45:44,831]     INFO   astrohack:  Checking parameter values for extract_pointing.extract_pointing 
[2024-04-24 10:45:44,832]     INFO   astrohack:  Module path: /export/home/fornax/Development/astrohack-issue-246/ 
[2024-04-24 10:45:44,833]     INFO   astrohack:  Searching /users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/graphviper/config/ for configuration file, please wait ... 
[2024-04-24 10:45:44,833]     INFO   astrohack:  Searching /export/home/fornax/Development/astrohack-issue-246/src/astrohack/config/ for configuration file, please wait ... 
Successful readonly open of usernoread-locked table /.lustre/cv/users/jhoskins/panel-tests/otf_holo_ka.60362.21125040509.ms/ANTENNA: 8 columns, 27 rows
[2024-04-24 10:46:04,056]     INFO   astrohack:  Finished processing 
[2024-04-24 10:46:04,266]     INFO   astrohack:  Checking parameter values for extract_holog.extract_holog 
[2024-04-24 10:46:04,267]     INFO   astrohack:  Module path: /export

In [104]:
pipeline.grid_search(n=10)

[2024-04-24 11:31:05,542]     INFO   astrohack:  Checking parameter values for panel.panel 
[2024-04-24 11:31:05,543]     INFO   astrohack:  Module path: /export/home/fornax/Development/astrohack-issue-246/ 
[2024-04-24 11:31:05,543]     INFO   astrohack:  Searching /users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/graphviper/config/ for configuration file, please wait ... 
[2024-04-24 11:31:05,544]     INFO   astrohack:  Searching /export/home/fornax/Development/astrohack-issue-246/src/astrohack/config/ for configuration file, please wait ... 
[2024-04-24 11:31:05,553]  WARNING   astrohack:  data/grid_search.panel.before.zarr will be overwritten. 
[2024-04-24 11:31:06,209]     INFO  astrohack_0:  processing ant_ea16 ddi_1 
[2024-04-24 11:31:06,210]     INFO  astrohack_0:  Searching /export/home/fornax/Development/astrohack-issue-246/src/astrohack for vla.zarr, please wait ... 
[2024-04-24 11:31:08,806]  WARNING  astrohack_0:  Fit failed with the full_parabo

/users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[2024-04-24 11:31:09,056]     INFO   astrohack:  Finished processing 
[2024-04-24 11:31:09,084]     INFO   astrohack:  Checking parameter values for panel.panel 
[2024-04-24 11:31:09,084]     INFO   astrohack:  Module path: /export/home/fornax/Development/astrohack-issue-246/ 
[2024-04-24 11:31:09,084]     INFO   astrohack:  Searching /users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/graphviper/config/ for configuration file, please wait ... 
[2024-04-24 11:31:09,085]     INFO   astrohack:  Searching /export/home/fornax/Development/astrohack-issue-246/src/astrohack/config/ for configuration file, please wait ... 
[2024-04-24 11:31:09,093]  WARNING   astrohack:  data/grid_search.panel.after.zarr will be overwritten. 
[2024-04-24 11:31:10,049]     INFO  astrohack_1:  processing ant_ea16 ddi_1 
[2024-04-24 11:31:10,049]     INFO  astrohack_1:  Searching /export/home/fornax/Development/astrohack-issue-246/src/astrohack for vla.zarr, please wait ... 
[2024-04-24 

/users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/users/jhoskins/fornax/miniconda3/envs/issue-246/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [105]:
df = pd.read_csv("grid_search.csv")
df

,clip_type,clip_level,panel_model,panel_margins,3-16.0,3-16.1,3-16.2,3-16.3,5-36.0,5-36.1,5-36.2,5-36.3,4-6.0,4-6.1,4-6.2,4-6.3,results
0,relative,2.0,full_paraboloid_lst_sq,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,372.727273


In [107]:
from astrohack.dio import open_panel

panel_mds = open_panel("data/grid_search.panel.after.zarr")
panel_mds["ant_ea16"]["ddi_1"]

<xarray.Dataset> Size: 136MB
Dimensions:             (u: 1492, v: 1492, labels: 172, pars: 1, screws: 4)
Coordinates:
  * labels              (labels) object 1kB '1-1' '1-2' '1-3' ... '6-39' '6-40'
  * pars                (pars) int64 8B 0
  * screws              (screws) <U2 32B 'il' 'ir' 'ol' 'or'
  * u                   (u) float64 12kB 13.74 13.72 13.7 ... -13.7 -13.72
  * v                   (v) float64 12kB -13.74 -13.72 -13.7 ... 13.7 13.72
Data variables:
    AMPLITUDE           (u, v) float64 18MB dask.array<chunksize=(187, 373), meta=np.ndarray>
    CORRECTIONS         (u, v) float64 18MB dask.array<chunksize=(187, 373), meta=np.ndarray>
    DEVIATION           (u, v) float64 18MB dask.array<chunksize=(187, 373), meta=np.ndarray>
    MASK                (u, v) bool 2MB dask.array<chunksize=(373, 746), meta=np.ndarray>
    PANEL_DISTRIBUTION  (u, v) int32 9MB dask.array<chunksize=(373, 373), meta=np.ndarray>
    PANEL_PARAMETERS    (labels, pars) float64 1kB dask.array<chunksize=(172, 1), meta=np.ndarray>
    PANEL_SCREWS        (labels, screws) float64 6kB dask.array<chunksize=(172, 4), meta=np.ndarray>
    PHASE               (u, v) float64 18MB dask.array<chunksize=(187, 373), meta=np.ndarray>
    PHASE_CORRECTIONS   (u, v) float64 18MB dask.array<chunksize=(187, 373), meta=np.ndarray>
    PHASE_RESIDUALS     (u, v) float64 18MB dask.array<chunksize=(187, 373), meta=np.ndarray>
    RESIDUALS           (u, v) float64 18MB dask.array<chunksize=(187, 373), meta=np.ndarray>
Attributes: (12/16)
    amp_unit:             V
    ant_name:             ea16
    aperture_resolution:  [0.7470588235294118, 0.7470588235294118]
    clip:                 8.197946797111069
    ddi:                  ddi_1
    fitted:               True
    ...                   ...
    panel_margin:         0.05
    panel_model:          full_paraboloid_lst_sq
    solved:               True
    telescope_name:       VLA
    theoretical_gain:     102.70870570627997
    wavelength:           0.008995482881747532

In [94]:
'''
from astrohack.panel import panel

panel_model = 'rigid'

panel_mds = panel(
    image_name="/.lustre/cv/users/jhoskins/panel-tests/otf_holo_ka.60362.21125040509.ms.image.after.zarr",
    panel_name="/.lustre/cv/users/jhoskins/panel-tests/otf_holo_ka.60362.21125040509.ms.panel.after.zarr",
    clip_type='sigma',
    clip_level=0.0,
    panel_model="corotated_scipy",
    parallel=True,
    overwrite=True
)
'''

'\nfrom astrohack.panel import panel\n\npanel_model = \'rigid\'\n\npanel_mds = panel(\n    image_name="/.lustre/cv/users/jhoskins/panel-tests/otf_holo_ka.60362.21125040509.ms.image.after.zarr",\n    panel_name="/.lustre/cv/users/jhoskins/panel-tests/otf_holo_ka.60362.21125040509.ms.panel.after.zarr",\n    clip_type=\'sigma\',\n    clip_level=0.0,\n    panel_model="corotated_scipy",\n    parallel=True,\n    overwrite=True\n)\n'

In [ ]:
client.shutdown()